# Custom Environment with RL

## Import dependencies

In [2]:
import os
import gym
from gym import Env
from gym.spaces import Box, Discrete, Tuple, Dict, MultiBinary, MultiDiscrete
import numpy as np
import random

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Types of Spaces

**There are six types of spaces supported by OpenAIgym**..

**1. Box** - A n dimentional box where each co ordinate lies between a bound define by [low, high]

**2. Discrete** - The space consists of n distinct points each mapped to an integer value in the interval[0, n-1]

**3. Tuple** - Tuple is a product of sample space..
>> **gym.spaces.Tuple((gym.spaces.Discrete(2), gym.spaces.Discrete(3))**

**4. Dict** - A Dict of sample Spaces types to make a complex space..
>> **gym.spaces.Dict({"position":gym.spaces.Discrete(2), "velocity":gym.spaces.Discrte(3)})**

**5. MultiBinary** - Define an n-dimensional binary spaces where the argument to MultiBinary() defines the number of n Dimension..
>> **gym.space.MultiBinary(6)**

**6. MultiDiscrete** - Multi dimensionalDiscrete Space, consists of a series of discrete action spaces with different number of action in each.. Parameterized by passing an array of positive integer for the number of actions for each discrete action space..
>> **gym.spaces.MultiDiscrete([5,2,3])**...


In [4]:
Discrete(3)

Discrete(3)

In [8]:
Discrete(3).sample()

1

In [11]:
Box(0,1,shape = (3,3))

Box([[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]], [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]], (3, 3), float32)

In [9]:
Box(0,1, shape = (3,3)).sample()

array([[0.29917297, 0.68896836, 0.48935655],
       [0.8216352 , 0.37601402, 0.24030143],
       [0.5879358 , 0.07662372, 0.43027392]], dtype=float32)

In [13]:
Box(0,255, shape = (2,2),dtype = int).sample()

array([[ 49, 143],
       [ 85, 226]])

In [16]:
Tuple((Box(0,1,shape = (3,3)), Discrete(2)))

Tuple(Box([[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]], [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]], (3, 3), float32), Discrete(2))

In [19]:
Tuple((Discrete(2), Box(1,2,shape = (2,1)))).sample()

(1,
 array([[1.2165664],
        [1.1913924]], dtype=float32))

In [21]:
Dict({"Height":Discrete(2), "Speed":Box(0,1,shape = (1,))})

Dict(Height:Discrete(2), Speed:Box([0.], [1.], (1,), float32))

In [22]:
Dict({"Height":Discrete(2), "Speed":Box(0,1,shape = (1,))}).sample()

OrderedDict([('Height', 0), ('Speed', array([0.92578584], dtype=float32))])

In [23]:
MultiBinary(5)

MultiBinary(5)

In [24]:
MultiBinary(6).sample()

array([1, 0, 0, 0, 0, 1], dtype=int8)

In [25]:
MultiDiscrete([4,4,3])

MultiDiscrete([4 4 3])

In [30]:
MultiDiscrete([4,5,1]).sample()

array([2, 0, 0])

# Building An Environment

In this Environment we are gonna build an agent which gives us best shower possible..By Randomly choosen Tempareture ..Tempareture range will be 37 to 39 degree..

In [37]:
class ShowerEnv(Env):
    
    def __init__(self):
        
        #Action - take,down,stay,up
        self.action_space = Discrete(3)
        
        #Temperature array
        self.observation_space = Box(low = 0, high = 100, shape = (1,))
        
        #set Start temperature
        self.state = 38 + random.randint(-3,3) 
        
        #set shower length 
        self.shower_length = 100
        
    def step(self, action):
        #Apply action
        self.state += action - 1
        
        #reduce shower length by 1 seconds
        self.shower_length -= 1
        
        #calculate Reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1
            
        #check if shower is done
        if self.shower_length <= 0:
            done = True
        else:
            done = False
            
        info = {}
        
        return self.state, reward, done, info
    
    def render(self):
        pass
    
    def reset(self):
        
        #reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        
        #reset shower time
        self.shower_length = 60
        
        return self.state

In [38]:
env = ShowerEnv()

In [41]:
env.observation_space.sample()

array([55.745857], dtype=float32)

In [42]:
env.reset()

array([35.])

In [43]:
from stable_baselines3.common.env_checker import check_env

In [45]:
check_env(env,warn = True)

# Test Environment

In [46]:
episodes = 5

for ep in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state,reward,done,info = env.step(action)
        score += reward
    print(f"Episodes:{ep} Score:{score}")
    
env.close()

Episodes:1 Score:-34
Episodes:2 Score:4
Episodes:3 Score:-14
Episodes:4 Score:22
Episodes:5 Score:-60


# Train Model

In [47]:
log_path = os.path.join("Training model", "logs","Custom env log")


In [49]:
model = PPO("MlpPolicy", env, verbose = 1, tensorboard_log = log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [50]:
model.learn(total_timesteps = 50000)

2021-12-25 02:20:44.718520: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/robin/anaconda3/envs/robin/lib/python3.7/site-packages/cv2/../../lib64:
2021-12-25 02:20:44.718592: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Logging to Training model/logs/Custom env log/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -27.8    |
| time/              |          |
|    fps             | 92       |
|    iterations      | 1        |
|    time_elapsed    | 22       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -30         |
| time/                   |             |
|    fps                  | 162         |
|    iterations           | 2           |
|    time_elapsed         | 25          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008680293 |
|    clip_fraction        | 0.0227      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_varia

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -26.6       |
| time/                   |             |
|    fps                  | 427         |
|    iterations           | 11          |
|    time_elapsed         | 52          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.008283811 |
|    clip_fraction        | 0.128       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.06       |
|    explained_variance   | 0.00027     |
|    learning_rate        | 0.0003      |
|    loss                 | 33          |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.0109     |
|    value_loss           | 67.4        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -23.7       |
| time/                   |             |
|    fps                  | 513         |
|    iterations           | 21          |
|    time_elapsed         | 83          |
|    total_timesteps      | 43008       |
| train/                  |             |
|    approx_kl            | 0.005832602 |
|    clip_fraction        | 0.0625      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.791      |
|    explained_variance   | -0.00754    |
|    learning_rate        | 0.0003      |
|    loss                 | 33.1        |
|    n_updates            | 200         |
|    policy_gradient_loss | -0.00314    |
|    value_loss           | 58.9        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

# Save Model

In [51]:
save_path = os.path.join("Training model","save model","Shower model")

In [52]:
model.save(save_path)

# Evaluate model

In [57]:
evaluate_policy(model,env,n_eval_episodes = 10,render = False)

(-12.0, 58.787753826796276)